In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("../../data/prediksi/prediksi_inset_nostop.csv", sep=";")

In [3]:
data

,Tweet,label,Tweet_Parsed,Prediksi,Prediksi_Score
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",baterai negatif,admin normal ke kalau baru pakai beberapa ...,baterai negatif,baterai -4
1,Kmera iphone 11 mmeng beda,kamera positif,kamera memang beda,kamera positif,kamera 0
2,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera negatif,kamera depannya jelek aku liat liat para p...,kamera negatif,kamera -3
3,Apple meluncurkan iPhone 11 dengan peningkatan...,baterai positif,meluncurkan dengan peningkatan daya tahan ba...,baterai positif,baterai 3
4,"xs max, senderrr. tapi rugi sih kalo kamu beli...","spesifikasi positif,harga positif",tapi rugi kalau kamu beli baru sekarang ...,"spesifikasi negatif,harga negatif","spesifikasi -9,harga -1"
...,...,...,...,...,...
743,ga keren banget masa charger bawaannya belum f...,baterai negatif,tidak keren sekali masa charger bawaannya belu...,baterai negatif,baterai -6
744,"yaelah belum fast charging, lama nih kalo ngec...",baterai negatif,yaelah belum cepat charger lama kalau charge ...,baterai negatif,baterai -5
745,butuh waku lama buat ngecharge kalo pake charg...,baterai negatif,butuh waku lama buat charge kalau menggunakan ...,baterai negatif,baterai -8
746,udah mahal begini masa chargernya blm fast cha...,baterai negatif,udah mahal begini masa chargernya belum cepat ...,"harga positif,baterai negatif","harga 1,baterai -1"


In [4]:
class eval:
    def __init__(self):
        pass
    
    def aspect_count(self,data, aspect, label_col = 'label'):
        label_list = data[label_col].values
        pos, neg = 0, 0
        for i in label_list:
            split = i.split(",")
            for j in split:
                in_split = j.split(" ")
                if in_split[0] == aspect:
                    if in_split[1] == 'negatif':
                        neg = neg+1
                    else:
                        pos = pos+1
        return pos,neg
    
    def eval_count(self,data,true_label_col='label',pred_label_col='Prediksi',aspect_list=['general','spesifikasi','kamera','baterai','harga']):
        label_list = [str(i).split(",") for i in data[true_label_col].values]
        pred_list = [str(i).split(",") for i in data[pred_label_col].values]
        sent = ['positif','negatif']
        eval_list = []
        for i in aspect_list:
            for j in range(len(sent)):
                asp_count = self.aspect_count(data,i)
                record = [i,sent[j],asp_count[j]]
                pred_count = 0
                pred_true = 0
                pred_false = 0
                for k,l in zip(label_list,pred_list):
                    if " ".join([i,sent[j]]) in l:
                        pred_count = pred_count+1
                        if " ".join([i,sent[j]]) in k:
                            pred_true = pred_true+1
                        else:
                            pred_false = pred_false+1
                record.extend([pred_count,pred_true,pred_false])
                eval_list.append(record)
        return eval_list
    
    def get_pred_text(self,data,true_label_col='label',pred_label_col='Prediksi',text_col="Tweet",prep_text_col='Tweet_Parsed',predicted = False):
        label_list = data[true_label_col].values
        pred_list = data[pred_label_col].values
        text_list = data[text_col].values
        prep_text_list = data[prep_text_col].values
        pred_text = []
        for i in range(data.shape[0]):
            if predicted == False:
                if label_list[i] != pred_list[i]:
                    pred_text.append([text_list[i],prep_text_list[i],label_list[i],pred_list[i]])
            else:
                if label_list[i] == pred_list[i]:
                    pred_text.append([text_list[i],prep_text_list[i],label_list[i],pred_list[i]])
                
        return pred_text

In [5]:
evaluate = eval()

In [6]:
eval_result = evaluate.eval_count(data)
eval_result = pd.DataFrame(eval_result,columns=['aspect','sentiment','label count','predicted count','true predict','false predict'])
eval_result = eval_result.groupby(['aspect','sentiment']).sum()
eval_result['percent_true'] = round((eval_result['true predict']/eval_result['label count'])*100,2)
eval_result['percent_predict_true'] = round((eval_result['true predict']/eval_result['predicted count'])*100,2)
eval_result

label count  predicted count  true predict  \
aspect      sentiment                                               
baterai     negatif             77              119            58   
            positif             82               45            25   
general     negatif             82              152            56   
            positif            107               99            55   
harga       negatif             79               56            32   
            positif             65               85            42   
kamera      negatif             70               94            43   
            positif            110               82            53   
spesifikasi negatif             62               67            24   
            positif             86               49            31   

                       false predict  percent_true  percent_predict_true  
aspect      sentiment                                                     
baterai     negatif               61         75.32                 48.74  
            positif               20         30.49                 55.56  
general     negatif               96         68.29                 36.84  
            positif               44         51.40                 55.56  
harga       negatif               24         40.51                 57.14  
            positif               43         64.62                 49.41  
kamera      negatif               51         61.43                 45.74  
            positif               29         48.18                 64.63  
spesifikasi negatif               43         38.71                 35.82  
            positif               18         36.05                 63.27

In [7]:
false_tweet = evaluate.get_pred_text(data)
false_tweet = pd.DataFrame(false_tweet,columns=['Tweet','Tweet_Parsed','True_Label','Pred_Label'])
false_tweet

,Tweet,Tweet_Parsed,True_Label,Pred_Label
0,"xs max, senderrr. tapi rugi sih kalo kamu beli...",tapi rugi kalau kamu beli baru sekarang ...,"spesifikasi positif,harga positif","spesifikasi negatif,harga negatif"
1,Saya juga dulu confuse nak pilih iphone 11 or ...,saya juga dulu bingung ingin pilih atau re...,kamera negatif,"spesifikasi negatif,general positif,kamera neg..."
2,Kalo sender lebih mentingin ketajaman layar me...,kalau lebih lebih penting ketajaman layar leb...,"general positif,kamera positif,spesifikasi pos...","general positif,kamera positif,spesifikasi neg..."
3,Cantik babi purple iphone 11 ni sialla apahal ...,cantik ungu ini padahal aku tidak kaya,general positif,general negatif
4,Tidak murah sbb iphone 12 skrin sama besar dgn...,tidak murah karena layar sama besar dengan ...,"harga negatif,general positif","harga negatif,general negatif"
...,...,...,...,...
394,iphone11 ini menggunakan chipset terkuat dan t...,ini menggunakan chipset terkuat dan tercepat ...,baterai positif,"spesifikasi negatif,baterai negatif"
395,buat iphone 11 ini apple udah mengembangkan te...,buat ini udah mengembangkan teknologi khusus...,baterai positif,baterai negatif
396,ip11 ini punya teknologi yang bisa menekan pen...,ip ini punya teknologi bisa menekan penggunaa...,baterai positif,"baterai positif,spesifikasi negatif"
397,udah mahal begini masa chargernya blm fast cha...,udah mahal begini masa chargernya belum cepat ...,baterai negatif,"harga positif,baterai negatif"


In [8]:
true_tweet = evaluate.get_pred_text(data,predicted=True)
true_tweet = pd.DataFrame(true_tweet,columns=['Tweet','Tweet_Parsed','True_Label','Pred_Label'])
true_tweet

,Tweet,Tweet_Parsed,True_Label,Pred_Label
0,"Min, normal ke kalau iPhone 11/11 Pro /Pro Max...",admin normal ke kalau baru pakai beberapa ...,baterai negatif,baterai negatif
1,Kmera iphone 11 mmeng beda,kamera memang beda,kamera positif,kamera positif
2,Iphone 11 pro max kamera depannya jelek ye? Gu...,kamera depannya jelek aku liat liat para p...,kamera negatif,kamera negatif
3,Apple meluncurkan iPhone 11 dengan peningkatan...,meluncurkan dengan peningkatan daya tahan ba...,baterai positif,baterai positif
4,Hasil kamera xiaomi note 10 bisa di adulah sam...,hasil kamera xiaomi note bisa di adulah sama ...,kamera positif,kamera positif
...,...,...,...,...
344,"baterai nya boros, ga cocok buat orang yang pe...",baterai boros tidak cocok buat orang pekerja...,baterai negatif,baterai negatif
345,ngecharge nya butuh watu lama ini iphone 11,charge butuh watu lama ini,baterai negatif,baterai negatif
346,ga keren banget masa charger bawaannya belum f...,tidak keren sekali masa charger bawaannya belu...,baterai negatif,baterai negatif
347,"yaelah belum fast charging, lama nih kalo ngec...",yaelah belum cepat charger lama kalau charge ...,baterai negatif,baterai negatif


In [9]:
eval_result.to_csv("../../data/final/evalresult_inset_nostop.csv", sep=";", index=False)
false_tweet.to_csv("../../data/final/falsetweet_inset_nostop.csv", sep=";", index=False)
true_tweet.to_csv("../../data/final/truetweet_inset_nostop.csv", sep=";", index=False)